In [1]:
import numpy as np 
import pandas as pd
import copy
import sys
from astropy.io import fits
import plotly.graph_objects as go
from astropy.coordinates import SkyCoord, GalacticLSR, Galactic
import astropy.units as u
sys.path.append('../../dash_gaia_orbits/')

# Package imports
import dash_gaia_orbits
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

column_renaming_dict = {'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'}

A new version of galpy (1.10.0) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


In [2]:
import importlib
importlib.reload(dash_gaia_orbits)

<module 'dash_gaia_orbits' from '/Users/cam/Desktop/astro_research/radcliffe/traceback_orbits_package/dash_gaia_orbits/notebooks/../../dash_gaia_orbits/dash_gaia_orbits/__init__.py'>

In [3]:
time_int = np.round(np.arange(0., -60.5, -0.5), 1)

In [4]:
dfme = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
dfme = dfme.drop(columns=['x', 'y', 'z'])
dfme = dfme[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars', 'mass_50']]
dfme = dfme.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

df_ap = dfme.loc[dfme['name'] == 'Melotte_20']
df_apfam = dfme.loc[dfme['family'] == 'alphaPer']
dfsun = pd.DataFrame({'name' : ['Sun'], 'age_myr' : [100000], 'U' : [0], 'V' : [0], 'W' : [0], 'x' : [0], 'y' : [0], 'z' : [0], 'n_stars' : [1]})

In [5]:
static_traces = []
static_traces_times = []

dfv = pd.read_csv('/Users/cam/Desktop/astro_research/radcliffe/mustache_work/figures/cubes/edenhofer_xyz_4pc.csv')
dfv = dfv.loc[dfv['e'].between(0.003, 1)]
dfv = dfv.loc[
    (dfv['x'].between(df_ap['x'].iloc[0] - 1000, df_ap['x'].iloc[0] + 1000)) &
    (dfv['y'].between(df_ap['y'].iloc[0] - 1000, df_ap['y'].iloc[0] + 1000)) &
    (dfv['z'].between(df_ap['z'].iloc[0] - 1000, df_ap['z'].iloc[0] + 1000))
]
ds_index = 1
scatter_edenhofer = go.Scatter3d(
    x=dfv['x'].values[::ds_index],
    y=dfv['y'].values[::ds_index],
    z=dfv['z'].values[::ds_index],
    mode='markers',
    marker=dict(size=.5,
                #color='gray',
                symbol='circle',
                opacity=.8),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0]) # only show at t=0

In [6]:
ap_fam = StarClusterData(df_apfam, data_name = 'Alpha Persei Family', min_size = 2, max_size = 25, color = 'violet', opacity = 1., marker_style = 'circle', show_tracks=False, size_by_n_stars=True)
# ap = StarClusterData(df_ap, data_name = 'Alpha Persei Cluster', min_size = 0, max_size = 18, color = 'violet', opacity = 1., marker_style = 'circle', show_tracks = True)
# sun = StarClusterData(dfsun, data_name = 'Sun', min_size = 0, max_size = 10, color = 'yellow', opacity = 1, marker_style = 'circle', show_tracks = False)

sc = StarClusterCollection([ap_fam])
#sc = StarClusterCollection([ap, ap_fam, sun])




xyz_widths = (1001, 1001, 201)
# xyz_ranges = [
#     (df_ap['x'].iloc[0] - 400, df_ap['x'].iloc[0] + 400),
#     (df_ap['y'].iloc[0] - 400, df_ap['y'].iloc[0] + 400),
#     (df_ap['z'].iloc[0] - 400, df_ap['z'].iloc[0] + 400)
# ]

plot_3d = StarClusters3DPlotter(
    data_collection = sc, 
    xyz_widths = xyz_widths, 
    #xyz_ranges = xyz_ranges,
    figure_theme = 'dark', 
    trace_grouping_dict=None)

save_name = '/Users/cam/Downloads/alpha_per.html'

fig3d_m6 = plot_3d.generate_3d_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    static_traces = static_traces, 
    static_traces_times = static_traces_times, 
    static_traces_legendonly=False,
    show_gc_line=False,
    #focus_group = 'Alpha Persei Family',
    focus_group = None,
    fade_in_time = 10, # Myr,
    fade_in_and_out = False
)